In [264]:
import pandas as pd
import numpy as np
import plotly.express as px  
import re
from statistics import mean 

In [265]:
data = pd.read_csv('data/arabica_data_cleaned.csv')

In [266]:
data.columns

Index(['Unnamed: 0', 'Species', 'Owner', 'Country.of.Origin', 'Farm.Name',
       'Lot.Number', 'Mill', 'ICO.Number', 'Company', 'Altitude', 'Region',
       'Producer', 'Number.of.Bags', 'Bag.Weight', 'In.Country.Partner',
       'Harvest.Year', 'Grading.Date', 'Owner.1', 'Variety',
       'Processing.Method', 'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body',
       'Balance', 'Uniformity', 'Clean.Cup', 'Sweetness', 'Cupper.Points',
       'Total.Cup.Points', 'Moisture', 'Category.One.Defects', 'Quakers',
       'Color', 'Category.Two.Defects', 'Expiration', 'Certification.Body',
       'Certification.Address', 'Certification.Contact', 'unit_of_measurement',
       'altitude_low_meters', 'altitude_high_meters', 'altitude_mean_meters'],
      dtype='object')

In [267]:
data = data.drop('Unnamed: 0', axis=1)

In [268]:
data[['Species', 'Country.of.Origin', 'Altitude', 'Region', 'Harvest.Year', 'Variety', 'Processing.Method', 'Aroma', 'Flavor', 'Aftertaste', 'Acidity', 'Body', 'Balance', 'Uniformity', 'Clean.Cup', 'Sweetness', 'Cupper.Points', 'Total.Cup.Points']]

,Species,Country.of.Origin,Altitude,Region,Harvest.Year,Variety,Processing.Method,Aroma,Flavor,Aftertaste,Acidity,Body,Balance,Uniformity,Clean.Cup,Sweetness,Cupper.Points,Total.Cup.Points
0,Arabica,Ethiopia,1950-2200,guji-hambela,2014,NaN,Washed / Wet,8.67,8.83,8.67,8.75,8.50,8.42,10.00,10.00,10.00,8.75,90.58
1,Arabica,Ethiopia,1950-2200,guji-hambela,2014,Other,Washed / Wet,8.75,8.67,8.50,8.58,8.42,8.42,10.00,10.00,10.00,8.58,89.92
2,Arabica,Guatemala,1600 - 1800 m,NaN,NaN,Bourbon,NaN,8.42,8.50,8.42,8.42,8.33,8.42,10.00,10.00,10.00,9.25,89.75
3,Arabica,Ethiopia,1800-2200,oromia,2014,NaN,Natural / Dry,8.17,8.58,8.42,8.42,8.50,8.25,10.00,10.00,10.00,8.67,89.00
4,Arabica,Ethiopia,1950-2200,guji-hambela,2014,Other,Washed / Wet,8.25,8.50,8.25,8.50,8.42,8.33,10.00,10.00,10.00,8.58,88.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,Arabica,Mexico,900,juchique de ferrer,2012,Bourbon,Washed / Wet,7.08,6.83,6.25,7.42,7.25,6.75,10.00,0.00,10.00,6.75,68.33
1307,Arabica,Haiti,~350m,"department d'artibonite , haiti",2012,Typica,Natural / Dry,6.75,6.58,6.42,6.67,7.08,6.67,9.33,6.00,6.00,6.42,67.92
1308,Arabica,Nicaragua,1100,jalapa,2016,Caturra,Other,7.25,6.58,6.33,6.25,6.42,6.08,6.00,6.00,6.00,6.17,63.08
1309,Arabica,Guatemala,4650,nuevo oriente,2012,Catuai,Washed / Wet,7.50,6.67,6.67,7.67,7.33,6.67,8.00,1.33,1.33,6.67,59.83


In [269]:
acidity_by_country = data[['Country.of.Origin', 'Acidity']].groupby(['Country.of.Origin']).mean().reset_index()

In [270]:
fig = px.bar(acidity_by_country, x="Country.of.Origin", y="Acidity")  
fig.show()

Let's try to predict if the coffee is outstanding, excellent, very good or not speciality coffee whatsoever (grading from SCA): 

Total Score Quality Classification

90-100 - Outstanding - Specialty
85-99.99 - Excellent - Specialty
80-84.99 - Very Good - Specialty
< 80.0 - Below Specialty Quality - Not Specialty

based on initial input data about the coffee sample like species, country of origin, altitude, region, etc. 

Let's pick the predictors that we will use and drop the irrelevant columns. 

In [350]:
coffee_df = data[['Species', 'Country.of.Origin', 'Altitude', 'Region', 'Harvest.Year', 'Variety', 'Processing.Method', 'Total.Cup.Points']]
coffee_df

,Species,Country.of.Origin,Altitude,Region,Harvest.Year,Variety,Processing.Method,Total.Cup.Points
0,Arabica,Ethiopia,1950-2200,guji-hambela,2014,NaN,Washed / Wet,90.58
1,Arabica,Ethiopia,1950-2200,guji-hambela,2014,Other,Washed / Wet,89.92
2,Arabica,Guatemala,1600 - 1800 m,NaN,NaN,Bourbon,NaN,89.75
3,Arabica,Ethiopia,1800-2200,oromia,2014,NaN,Natural / Dry,89.00
4,Arabica,Ethiopia,1950-2200,guji-hambela,2014,Other,Washed / Wet,88.83
...,...,...,...,...,...,...,...,...
1306,Arabica,Mexico,900,juchique de ferrer,2012,Bourbon,Washed / Wet,68.33
1307,Arabica,Haiti,~350m,"department d'artibonite , haiti",2012,Typica,Natural / Dry,67.92
1308,Arabica,Nicaragua,1100,jalapa,2016,Caturra,Other,63.08
1309,Arabica,Guatemala,4650,nuevo oriente,2012,Catuai,Washed / Wet,59.83


In [351]:
coffee_df.describe(include=['O']).T

,count,unique,top,freq
Species,1311,1,Arabica,1311
Country.of.Origin,1310,36,Mexico,236
Altitude,1088,383,1100,43
Region,1254,343,huila,112
Harvest.Year,1264,46,2012,352
Variety,1110,29,Caturra,256
Processing.Method,1159,5,Washed / Wet,812


Altitude variable has very different values format so let's try to clean it up so that we have just one altitude number (not a range) and no letters indicating metric system or ~ signs. 

In [352]:
coffee_df['Altitude'].replace(np.NaN, '', inplace=True)

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/pandas/core/series.py:4485: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [353]:
coffee_df['Altitude'] = coffee_df['Altitude'].str.findall(r"\d+")

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [354]:
def clean_altitude(x):
    if len(x) > 1:
        return mean(map(int, x)) 
    elif len(x) == 1:
        return x[0]
    return x
coffee_df['Altitude'] = coffee_df['Altitude'].apply(clean_altitude)

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [355]:
coffee_df['Altitude'] = pd.to_numeric(coffee_df['Altitude'])

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [356]:
coffee_df['Harvest.Year'].unique()

array(['2014', nan, '2013', '2012', 'March 2010',
       'Sept 2009 - April 2010', 'May-August', '2009/2010', '2015',
       '2011', '2016', '2015/2016', '2010', 'Fall 2009', '2017',
       '2009 / 2010', '2010-2011', '2009-2010', '2009 - 2010',
       '2013/2014', '2017 / 2018', 'mmm', 'TEST',
       'December 2009-March 2010', '2014/2015', '2011/2012',
       'January 2011', '4T/10', '2016 / 2017', '23 July 2010',
       'January Through April', '1T/2011', '4t/2010', '4T/2010',
       'August to December', 'Mayo a Julio', '47/2010', 'Abril - Julio',
       '4t/2011', 'Abril - Julio /2011', 'Spring 2011 in Colombia.',
       '3T/2011', '2016/2017', '1t/2011', '2018', '4T72010', '08/09 crop'],
      dtype=object)

Let's do the same thing with Harvest.Year, since we also have very nonuniform values there. 

In [357]:
coffee_df['Harvest.Year'].replace(np.NaN, '', inplace=True)

In [358]:
coffee_df['Harvest.Year'] = coffee_df['Harvest.Year'].str.findall(r"\d+")

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [359]:
def clean_harvest_year(x):
    if len(x) > 1:
        return mean(map(int, x)) 
    elif len(x) == 1:
        return x[0]
    return x
coffee_df['Harvest.Year'] = coffee_df['Harvest.Year'].apply(clean_harvest_year)

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [360]:
coffee_df['Harvest.Year'] = pd.to_numeric(coffee_df['Harvest.Year'])

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [361]:
coffee_df['Harvest.Year'].isna().sum()

57

In [362]:
coffee_df.dtypes

Species               object
Country.of.Origin     object
Altitude             float64
Region                object
Harvest.Year         float64
Variety               object
Processing.Method     object
Total.Cup.Points     float64
dtype: object

In [363]:
coffee_df.isna().sum()

Species                0
Country.of.Origin      1
Altitude             226
Region                57
Harvest.Year          57
Variety              201
Processing.Method    152
Total.Cup.Points       0
dtype: int64

We have quite some NaN values across the variables, that we should first get rid of by substituing them with the most common value in the category (grading). For that let's code the Total.Cup.Points according to Total Score Quality Classification described above. 

In [364]:
def code_grades(x):
    if x >= 90:
        return 'Outstanding' 
    elif x <= 89.99 and x >= 85:
        return 'Excellent'
    elif x <= 84.99 and x >= 80:
        return 'Very Good'
    else:
        return 'Not Specialty'
coffee_df['Total.Cup.Points'] = coffee_df['Total.Cup.Points'].apply(code_grades)

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [365]:
coffee_df.groupby(['Total.Cup.Points']).agg(lambda x:x.value_counts(dropna=False))

,Species
Total.Cup.Points,
Excellent,105
Not Specialty,181
Outstanding,1
Very Good,1024


In [366]:
coffee_df[coffee_df['Total.Cup.Points'] == 'Outstanding']

,Species,Country.of.Origin,Altitude,Region,Harvest.Year,Variety,Processing.Method,Total.Cup.Points
0,Arabica,Ethiopia,2075.0,guji-hambela,2014.0,NaN,Washed / Wet,Outstanding


We have just one case in the whole dataset, that the coffee has been graded 'Outstanding', so there is no way to learn anything about this grade. To avoid problems and additional work when getting rid of NaN values, let's just drop this entry and concentrate on three other grades. 

In [367]:
coffee_df.drop(0, inplace = True)

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Let's see if we can derive the Country of Origin from Region maybe for the one entry where Country of Origin is 0. 

In [368]:

coffee_df.loc[pd.isnull(coffee_df['Country.of.Origin'])]

,Species,Country.of.Origin,Altitude,Region,Harvest.Year,Variety,Processing.Method,Total.Cup.Points
1197,Arabica,NaN,NaN,NaN,NaN,NaN,NaN,Not Specialty


Unfortunately, most values are NaN for this entry, so we can just drop this row, since it doesn't convey any valuable information for the prediction. 

In [369]:
coffee_df.drop(1197, inplace = True)

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/pandas/core/frame.py:4312: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Now let's substitute NaNs in the Altitude and Harvest.Year column based on the most common values for this grade. 

In [370]:
coffee_df.groupby("Total.Cup.Points").median()

,Altitude,Harvest.Year
Total.Cup.Points,,
Excellent,1600.0,2013.0
Not Specialty,1280.0,2013.0
Very Good,1400.0,2014.0


In [371]:
coffee_features_numerical = coffee_df[['Altitude', 'Harvest.Year']]
for feature in coffee_features_numerical:
    value_for_excellent = coffee_df.groupby("Total.Cup.Points")[feature].median()[0]
    value_for_not_speciality = coffee_df.groupby("Total.Cup.Points")[feature].median()[1]
    value_for_very_good = coffee_df.groupby("Total.Cup.Points")[feature].median()[2]
    coffee_df.loc[(coffee_df["Total.Cup.Points"] == 'Excellent') & (coffee_df[feature].isnull()), feature] = value_for_excellent
    coffee_df.loc[(coffee_df["Total.Cup.Points"] == 'Not Specialty') & (coffee_df[feature].isnull()), feature] = value_for_not_speciality
    coffee_df.loc[(coffee_df["Total.Cup.Points"] == 'Very Good') & (coffee_df[feature].isnull()), feature] = value_for_very_good
print(coffee_df.isnull().sum())

Species                0
Country.of.Origin      0
Altitude               0
Region                56
Harvest.Year           0
Variety              199
Processing.Method    151
Total.Cup.Points       0
dtype: int64


/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/pandas/core/indexing.py:1719: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



We have NaN values left in columns Region, Variety and Processing method, that we have to take care of. It is okay to take most freuquent values for the category(grade) for Variety and Processing method, however we wouldn't do that for region, since we also have column country and they have to align. Let's start with variety and processing method. 

In [372]:
coffee_features_categorical = coffee_df[['Variety', 'Processing.Method']]
for feature in coffee_features_categorical:
    value_for_excellent = coffee_df[[feature, 'Total.Cup.Points']].groupby(['Total.Cup.Points']).agg(lambda x:x.value_counts(dropna=True).index[0]).reset_index().loc[0][feature]
    value_for_not_speciality = coffee_df[[feature, 'Total.Cup.Points']].groupby(['Total.Cup.Points']).agg(lambda x:x.value_counts(dropna=True).index[0]).reset_index().loc[1][feature]
    value_for_very_good = coffee_df[[feature, 'Total.Cup.Points']].groupby(['Total.Cup.Points']).agg(lambda x:x.value_counts(dropna=True).index[0]).reset_index().loc[2][feature]
    coffee_df.loc[(coffee_df["Total.Cup.Points"] == 'Excellent') & (coffee_df[feature].isnull()), feature] = value_for_excellent
    coffee_df.loc[(coffee_df["Total.Cup.Points"] == 'Not Specialty') & (coffee_df[feature].isnull()), feature] = value_for_not_speciality
    coffee_df.loc[(coffee_df["Total.Cup.Points"] == 'Very Good') & (coffee_df[feature].isnull()), feature] = value_for_very_good
print(coffee_df.isnull().sum())

Species               0
Country.of.Origin     0
Altitude              0
Region               56
Harvest.Year          0
Variety               0
Processing.Method     0
Total.Cup.Points      0
dtype: int64


In [373]:
most_frequent_region = coffee_df.groupby('Country.of.Origin')['Region'].apply(lambda x: x.value_counts().index[0]).reset_index()

In [374]:
nan_region = coffee_df[['Country.of.Origin', 'Region']][coffee_df['Region'].isnull()]
for nan in nan_region.iterrows():
    nan[1][1] = most_frequent_region[most_frequent_region['Country.of.Origin'] == nan[1][0]]['Region'].values[0]
nan_region

,Country.of.Origin,Region
2,Guatemala,oriente
5,Brazil,south of minas
6,Peru,san ignacio
30,Nicaragua,jinotega
31,"Tanzania, United Republic Of",arusha
100,Kenya,nyeri
114,Kenya,nyeri
134,Costa Rica,tarrazu
152,Kenya,nyeri
223,Guatemala,oriente


In [375]:
coffee_df.loc[pd.isnull(coffee_df['Region']), 'Region'] = nan_region['Region']

/Users/t.shears/.local/share/virtualenvs/coffee_quality-vjRsiDuH/lib/python3.7/site-packages/pandas/core/indexing.py:1675: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [376]:
coffee_df.isnull().sum()

Species              0
Country.of.Origin    0
Altitude             0
Region               0
Harvest.Year         0
Variety              0
Processing.Method    0
Total.Cup.Points     0
dtype: int64

In [377]:
coffee_df

,Species,Country.of.Origin,Altitude,Region,Harvest.Year,Variety,Processing.Method,Total.Cup.Points
1,Arabica,Ethiopia,2075.0,guji-hambela,2014.0,Other,Washed / Wet,Excellent
2,Arabica,Guatemala,1700.0,oriente,2013.0,Bourbon,Washed / Wet,Excellent
3,Arabica,Ethiopia,2000.0,oromia,2014.0,Caturra,Natural / Dry,Excellent
4,Arabica,Ethiopia,2075.0,guji-hambela,2014.0,Other,Washed / Wet,Excellent
5,Arabica,Brazil,1600.0,south of minas,2013.0,Caturra,Natural / Dry,Excellent
...,...,...,...,...,...,...,...,...
1306,Arabica,Mexico,900.0,juchique de ferrer,2012.0,Bourbon,Washed / Wet,Not Specialty
1307,Arabica,Haiti,350.0,"department d'artibonite , haiti",2012.0,Typica,Natural / Dry,Not Specialty
1308,Arabica,Nicaragua,1100.0,jalapa,2016.0,Caturra,Other,Not Specialty
1309,Arabica,Guatemala,4650.0,nuevo oriente,2012.0,Catuai,Washed / Wet,Not Specialty
